# Lbl2Vec Model

## LDA Topic Inference

Unsupervised text classification model. Requires sort of a dataset that can map terms to the correct clusters.
Can be done manually during the training stage of the LDA. But implementing this in the case where it might out-perform human input (such as long topic lists).
There has also been a paper showing that such models can perform better than LDAs; so if effective, the Lbl2vec model can be used for topic modeling directly.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups     # the dataset

In [2]:
# vars here

In [3]:
# load data
train = fetch_20newsgroups(subset='train', shuffle=False)
test = fetch_20newsgroups(subset='test', shuffle=False)

# parse data to pandas DataFrames
newsgroup_train = pd.DataFrame({'article':train.data, 'class_index':train.target})
newsgroup_test = pd.DataFrame({'article':test.data, 'class_index':test.target})

# load labels with keywords
labels = pd.read_csv('20newsgroups_keywords.csv',sep=';')

In [4]:
# data pre-processing

# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

# lets check our keywords
print(labels)

   class_index          class_name               keywords  number_of_keywords
0            8     rec.motorcycles    [bikes, motorcycle]                   2
1            9  rec.sport.baseball             [baseball]                   1
2           10    rec.sport.hockey               [hockey]                   1
3           11           sci.crypt  [encryption, privacy]                   2


In [5]:

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags
from gensim.models.doc2vec import TaggedDocument

# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

# add data set type column
newsgroup_train['data_set_type'] = 'train'
newsgroup_test['data_set_type'] = 'test'

# concat train and test data
newsgroup_full_corpus = pd.concat([newsgroup_train,newsgroup_test]).reset_index(drop=True)

# reduce dataset to only articles that belong to classes where we defined our keywords
newsgroup_full_corpus = newsgroup_full_corpus[newsgroup_full_corpus['class_index'].isin(list(labels['class_index']))]

# tokenize and tag documents for Lbl2Vec training
newsgroup_full_corpus['tagged_docs'] = newsgroup_full_corpus.apply(lambda row: TaggedDocument(tokenize(row['article']), 
    [str(row.name)]), axis=1)

# add doc_key column
newsgroup_full_corpus['doc_key'] = newsgroup_full_corpus.index.astype(str)

# add class_name column
newsgroup_full_corpus = newsgroup_full_corpus.merge(labels, left_on='class_index', right_on='class_index', how='left')
print(newsgroup_full_corpus.head())

                                             article  class_index   
0  From: cubbie@garnet.berkeley.edu (            ...            9  \
1  From: crypt-comments@math.ncsu.edu\nSubject: C...           11   
2  From: george@minster.york.ac.uk\nSubject: Non-...           11   
3  From: williac@govonca.gov.on.ca (Chris William...           10   
4  From: ayari@judikael.loria.fr (Ayari Iskander)...           10   

  data_set_type                                        tagged_docs doc_key   
0         train  ([from, cubbie, garnet, berkeley, edu, subject...       0  \
1         train  ([from, crypt, comments, math, ncsu, edu, subj...       2   
2         train  ([from, george, minster, york, ac, uk, subject...      11   
3         train  ([from, williac, govonca, gov, on, ca, chris, ...      12   
4         train  ([from, ayari, judikael, loria, fr, ayari, isk...      15   

           class_name               keywords  number_of_keywords  
0  rec.sport.baseball             [baseball]     

In [6]:
from lbl2vec import Lbl2Vec

# init model with parameters
Lbl2Vec_model = Lbl2Vec(keywords_list=list(labels.keywords), tagged_documents=newsgroup_full_corpus['tagged_docs'][newsgroup_full_corpus['data_set_type'] == 'train'], label_names=list(labels.class_name), similarity_threshold=0.43, min_num_docs=100, epochs=10)

# train model
Lbl2Vec_model.fit()

2023-11-06 15:23:06,383 - Lbl2Vec - INFO - Train document and word embeddings
2023-11-06 15:23:11,240 - Lbl2Vec - INFO - Train label embeddings


In [7]:
from sklearn.metrics import f1_score

# predict similarity scores
model_docs_lbl_similarities = Lbl2Vec_model.predict_model_docs()

# merge DataFrames to compare the predicted and true category labels
evaluation_train = model_docs_lbl_similarities.merge(newsgroup_full_corpus[newsgroup_full_corpus['data_set_type'] == 'train'], left_on='doc_key', right_on='doc_key')
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']
print(y_pred_train)

print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

2023-11-06 15:23:11,349 - Lbl2Vec - INFO - Get document embeddings from model


2023-11-06 15:23:11,367 - Lbl2Vec - INFO - Calculate document<->label similarities


0       rec.sport.baseball
1                sci.crypt
2                sci.crypt
3       rec.sport.baseball
4         rec.sport.hockey
               ...        
2385             sci.crypt
2386      rec.sport.hockey
2387             sci.crypt
2388      rec.sport.hockey
2389       rec.motorcycles
Name: most_similar_label, Length: 2390, dtype: object
F1 score: 0.8966527196652719


In [8]:
# predict similarity scores of new test documents (they were not used during Lbl2Vec training)
new_docs_lbl_similarities = Lbl2Vec_model.predict_new_docs(tagged_docs=newsgroup_full_corpus['tagged_docs'][newsgroup_full_corpus['data_set_type']=='test'])

# merge DataFrames to compare the predicted and true topic labels
evaluation_test = new_docs_lbl_similarities.merge(newsgroup_full_corpus[newsgroup_full_corpus['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')
y_true_test = evaluation_test['class_name']
y_pred_test = evaluation_test['most_similar_label']

print('F1 score:',f1_score(y_true_test, y_pred_test, average='micro'))

2023-11-06 15:23:12,860 - Lbl2Vec - INFO - Calculate document embeddings
2023-11-06 15:23:13,792 - Lbl2Vec - INFO - Calculate document<->label similarities


F1 score: 0.8849056603773585
